In [1]:
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import numpy as np

In [ ]:
def clean_images(folder_path):
    valid_exts = ['.jpg', '.jpeg', '.png']
    for root, _, files in os.walk(folder_path):
        for file in files:
            path = os.path.join(root, file)
            try:
                if os.path.splitext(file)[-1].lower() not in valid_exts:
                    os.remove(path)
                else:
                    with Image.open(path) as img:
                        img.verify()
            except Exception as e:
                print(f"Removing {path}: {e}")
                os.remove(path)

clean_images('path/to/Data/train')
clean_images('path/to/Data/test')


In [ ]:
train_path = 'path/to/Data/train'
test_path = 'path/to/Data/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [5]:
model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=15
)


Epoch 1/15
898/898 [==============================] - 70s 77ms/step - loss: 1.6638 - accuracy: 0.3413 - val_loss: 1.5121 - val_accuracy: 0.4220
Epoch 2/15
898/898 [==============================] - 74s 82ms/step - loss: 1.4916 - accuracy: 0.4233 - val_loss: 1.3940 - val_accuracy: 0.4744
Epoch 3/15
898/898 [==============================] - 59s 66ms/step - loss: 1.4140 - accuracy: 0.4561 - val_loss: 1.3618 - val_accuracy: 0.4811
Epoch 4/15
898/898 [==============================] - 68s 76ms/step - loss: 1.3554 - accuracy: 0.4810 - val_loss: 1.3142 - val_accuracy: 0.4872
Epoch 5/15
898/898 [==============================] - 67s 75ms/step - loss: 1.3037 - accuracy: 0.5018 - val_loss: 1.2842 - val_accuracy: 0.5049
Epoch 6/15
898/898 [==============================] - 82s 91ms/step - loss: 1.2609 - accuracy: 0.5184 - val_loss: 1.2701 - val_accuracy: 0.5173
Epoch 7/15
898/898 [==============================] - 74s 82ms/step - loss: 1.2109 - accuracy: 0.5363 - val_loss: 1.2443 - val_accuracy:

In [6]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")

225/225 [==============================] - 5s 24ms/step - loss: 1.2846 - accuracy: 0.5397
Test Accuracy: 53.97%


In [7]:
class_labels = list(train_generator.class_indices.keys())

In [8]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    face = cv2.cvtColor(cv2.resize(frame, (48, 48)), cv2.COLOR_BGR2GRAY)
    face = face.reshape(1, 48, 48, 1) / 255.0

    prediction = model.predict(face)
    emotion = class_labels[np.argmax(prediction)]

    cv2.putText(frame, emotion, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
    cv2.imshow("Emotion Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step


In [9]:
model.save('emotion_model.h5')

C:\Users\BALAJI\anaconda3\envs\emotion-env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
model.save('emotion_model.keras')